In [ ]:
!pip install librosa
!pip install pysndfx
!pip install python_speech_features
!apt-get install sox
!pip install PyWavelets
!pip install pydub
!pip install noisereduce
!pip install webrtcvad

In [ ]:
import IPython
import pydub
import noisereduce
import pywt
import librosa
import librosa.display
import pandas as pd
import numpy as np
import scipy.signal
import math
import scipy as sp
from pysndfx import AudioEffectsChain
import python_speech_features
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from pylab import rcParams
from pydub import AudioSegment
from noisereduce import reduce_noise
import webrtcvad
import wave
from google.colab import drive
drive.mount('/content/drive')

# **SNR(Signal-to-noise ratio)**
the standard to calculate the ac rate of denoise

In [ ]:
def calculate_snr(original_audio, denoised_audio):
    # Ensure both audio signals have the same length
    min_length = min(len(original_audio), len(denoised_audio))
    original_audio = original_audio[:min_length]
    denoised_audio = denoised_audio[:min_length]

    # Calculate the power of the original audio
    power_original = np.sum(np.square(original_audio))

    # Calculate the power of the noise (residual after denoising)
    noise = original_audio - denoised_audio
    power_noise = np.sum(np.square(noise))

    # Calculate SNR in decibels
    snr = 10 * np.log10(power_original / power_noise)

    return snr

# **Load original audio**

In [ ]:
file_path = '/content/drive/MyDrive/F03.wav'
y, sr = librosa.load(file_path)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform of Original Audio')
IPython.display.Audio(data=y, rate=22055)

In [ ]:
S1 = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=64)
D1 = librosa.power_to_db(S1, ref=np.max)
librosa.display.specshow(D1, x_axis='time', y_axis='mel');

# **NR library denoise result**

In [ ]:
source_file='/content/drive/MyDrive/F03.wav'
#output_file='/content/drive/MyDrive/Colab Notebooks/term project/~noise CTT/'+str(i)+'CTT.wav'
y, sr = librosa.load(file_path)
y_reduced = reduce_noise(y,sr=sr)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y, sr=sr)
librosa.display.waveshow(y_reduced, sr=sr)
IPython.display.Audio(data=y_reduced, rate=22055)

In [ ]:
calculate_snr(y, y_reduced)

# **Double NR implement**
Using NR library twice, and obsevering its effect.

In [ ]:
y_reduced_twice = reduce_noise(y_reduced,sr=sr)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y, sr=sr)
librosa.display.waveshow(y_reduced, sr=sr)
librosa.display.waveshow(y_reduced_twice, sr=sr)
IPython.display.Audio(data=y_reduced_twice, rate=22055)

In [ ]:
calculate_snr(y, y_reduced_twice)

# **Tunning NR parameter**
Keep adjusting and experimenting until you achieve the desired balance between noise reduction and preserving low-frequency voice.

In [ ]:
y_reduced = reduce_noise(
    y=y,sr=sr,
    freq_mask_smooth_hz=1500,  # Experiment with different values
    time_constant_s=2.0,  # Experiment with different values
    thresh_n_mult_nonstationary=1,  # Experiment with different values
    sigmoid_slope_nonstationary=10,  # Experiment with different values
    prop_decrease=1  # Experiment with different values
)
y_reduced_mod = reduce_noise(
    y=y_reduced,sr=sr,
    freq_mask_smooth_hz=500,  # Experiment with different values
    time_constant_s=1,  # Experiment with different values
    thresh_n_mult_nonstationary=1.5,  # Experiment with different values
    sigmoid_slope_nonstationary=5,  # Experiment with different values
    prop_decrease=1  # Experiment with different values
)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
#librosa.display.waveshow(y, sr=sr)
librosa.display.waveshow(y, sr=sr)
librosa.display.waveshow(y_reduced, sr=sr)
librosa.display.waveshow(y_reduced_mod, sr=sr)
#IPython.display.Audio(data=y_reduced, rate=22055)
IPython.display.Audio(data=y_reduced_mod, rate=22055)

In [ ]:
calculate_snr(y, y_reduced)

In [ ]:
calculate_snr(y, y_reduced_mod)

# **Enhance Audio with double NR**
with highshelf

In [ ]:
apply_audio_effects = AudioEffectsChain().highshelf(gain=10.0, frequency=10000, slope=0.1)
y_enhanced = apply_audio_effects(y_reduced)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y_enhanced, sr=sr)
librosa.display.waveshow(y_reduced, sr=sr)
IPython.display.Audio(data=y_enhanced, rate=22055)

In [ ]:
calculate_snr(y, y_enhanced)

with lowshelf

In [ ]:
apply_audio_effects = AudioEffectsChain().lowshelf(gain=10.0, frequency=10000, slope=0.1)
y_enhanced = apply_audio_effects(y_reduced)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y_enhanced, sr=sr)
librosa.display.waveshow(y_reduced, sr=sr)
IPython.display.Audio(data=y_enhanced, rate=22055)

In [ ]:
calculate_snr(y, y_enhanced)

# **Spectral subtraction**

In [ ]:
import numpy as np

# Function for spectral subtraction
def spectral_subtraction(y, noise_estimation_factor=1.5):
    # Compute the spectrogram
    spec = librosa.stft(y)

    # Estimate the noise spectrum
    noise = np.mean(np.abs(spec[:, :int(spec.shape[1] / 5)]), axis=1)

    # Apply spectral subtraction
    spec -= noise[:, np.newaxis] * noise_estimation_factor
    spec = np.maximum(spec, 0)

    # Inverse transform to obtain the denoised signal
    y_denoised = librosa.istft(spec)

    return y_denoised

# Apply spectral subtraction for noise reduction
y_denoised = spectral_subtraction(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 2)
librosa.display.waveshow(y, sr=sr)
librosa.display.waveshow(y_denoised, sr=sr)
plt.title('Waveform of Audio')
IPython.display.Audio(data=y_denoised, rate=22055)

In [ ]:
calculate_snr(y, y_denoised)

In [ ]:
# Visualize the spectrogram of the denoised audio
librosa.display.specshow(librosa.amplitude_to_db(librosa.stft(y_denoised), ref=np.max), y_axis='log', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of the Denoised Audio')
plt.show()

# **Filtering low-frequencies**
As we noticed, low frequencies does not contribute to bird sounds, a first idea is to remove these low frequencies. A high pass filter helps in this task. Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html

In [ ]:
from scipy import signal
import random

def f_high(y,sr):
    b,a = signal.butter(10, 2000/(sr/2), btype='highpass')
    yf = signal.lfilter(b,a,y)
    return yf
yf = f_high(y, sr)
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(yf,sr=sr, axis='time');
IPython.display.Audio(data=yf, rate=22055)

In [ ]:
calculate_snr(y, yf)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=yf, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **PCEN**
PCEN has become a very useful strategy for acoustic event detection, and it has shown to perform better in such tasks as a frontend. Its idea is to perform non-linear compression on time-frequency channels.

I am using the example shown here: https://librosa.org/doc/latest/generated/librosa.pcen.html?highlight=pcen#librosa.pcen

In [ ]:
Dp = librosa.pcen(S1 * (2**31), sr=sr, gain=1.1, hop_length=512, bias=2, power=0.5, time_constant=0.8, eps=1e-06, max_size=2)
yp = librosa.feature.inverse.mel_to_audio(Dp)
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time')
librosa.display.waveshow(yp,sr=sr, axis='time')
IPython.display.Audio(data=yp, rate=22055)

In [ ]:
calculate_snr(y, yp)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=yp, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **Spectral Gating**
This is also a technique for noise reduction based on gates that monitor audio level. It is commonly used in music industry, and present in tools like Audacity (https://wiki.audacityteam.org/wiki/How_Audacity_Noise_Reduction_Works). I reproduced here the code made available by Tim Sainburg in his github (https://github.com/timsainb/noisereduce).

In [ ]:
import time
from datetime import timedelta as td


def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length=hop_length,win_length=win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)


def plot_spectrogram(signal, title):
    fig, ax = plt.subplots(figsize=(20, 4))
    cax = ax.matshow(
        signal,
        origin="lower",
        aspect="auto",
        cmap=plt.cm.seismic,
        vmin=-1 * np.max(np.abs(signal)),
        vmax=np.max(np.abs(signal)),
    )
    fig.colorbar(cax)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


def plot_statistics_and_filter(
    mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 4))
    plt_mean, = ax[0].plot(mean_freq_noise, label="Mean power of noise")
    plt_std, = ax[0].plot(std_freq_noise, label="Std. power of noise")
    plt_std, = ax[0].plot(noise_thresh, label="Noise threshold (by frequency)")
    ax[0].set_title("Threshold for mask")
    ax[0].legend()
    cax = ax[1].matshow(smoothing_filter, origin="lower")
    fig.colorbar(cax)
    ax[1].set_title("Filter for smoothing Mask")
    plt.show()


def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False,
):
    """Remove noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)
        visual (bool): Whether to plot the steps of the algorithm

    Returns:
        array: The recovered signal with noise subtracted

    """
    if verbose:
        start = time.time()
    # STFT over noise
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # convert to dB
    # Calculate statistics over noise
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    if verbose:
        print("STFT on noise:", td(seconds=time.time() - start))
        start = time.time()
    # STFT over signal
    if verbose:
        start = time.time()
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    if verbose:
        print("STFT on signal:", td(seconds=time.time() - start))
        start = time.time()
    # Calculate value to mask dB to
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    print(noise_thresh, mask_gain_dB)
    # Create a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    # calculate the threshold for each frequency/time bin
    db_thresh = np.repeat(
        np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
        np.shape(sig_stft_db)[1],
        axis=0,
    ).T
    # mask if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh
    if verbose:
        print("Masking:", td(seconds=time.time() - start))
        start = time.time()
    # convolve the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease
    if verbose:
        print("Mask convolution:", td(seconds=time.time() - start))
        start = time.time()
    # mask the signal
    sig_stft_db_masked = (
        sig_stft_db * (1 - sig_mask)
        + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask
    )  # mask real
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (
        1j * sig_imag_masked
    )
    if verbose:
        print("Mask application:", td(seconds=time.time() - start))
        start = time.time()
    # recover the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )
    if verbose:
        print("Signal recovery:", td(seconds=time.time() - start))
    if visual:
        plot_spectrogram(noise_stft_db, title="Noise")
    if visual:
        plot_statistics_and_filter(
            mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
        )
    if visual:
        plot_spectrogram(sig_stft_db, title="Signal")
    if visual:
        plot_spectrogram(sig_mask, title="Mask applied")
    if visual:
        plot_spectrogram(sig_stft_db_masked, title="Masked signal")
    if visual:
        plot_spectrogram(recovered_spec, title="Recovered spectrogram")
    return recovered_signal

In [ ]:
noise1 = y[5*sr:6*sr]
yg = removeNoise(audio_clip=y, noise_clip=noise1,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False)
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(yg,sr=sr, axis='time');
IPython.display.Audio(data=yg, rate=22055)

In [ ]:
calculate_snr(y, yg)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=yg, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **POWER DENOISE**
receives an audio matrix, returns the matrix after gain reduction on noise

In [ ]:
cent = librosa.feature.spectral_centroid(y=y, sr=sr)
threshold_h = round(np.median(cent))*1.5
threshold_l = round(np.median(cent))*0.1
less_noise = AudioEffectsChain().lowshelf(gain=-30.0, frequency=threshold_l, slope=0.8).highshelf(gain=-12.0, frequency=threshold_h, slope=0.5)#.limiter(gain=6.0)
y_clean = less_noise(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_clean,sr=sr, axis='time');
IPython.display.Audio(data=y_clean, rate=22055)

In [ ]:
calculate_snr(y, y_clean)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=y_clean, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **Centroid denoise**
receives an audio matrix,
returns the matrix after gain reduction on noise

In [ ]:
cent = librosa.feature.spectral_centroid(y=y, sr=sr)
threshold_h = np.max(cent)
threshold_l = np.min(cent)
less_noise = AudioEffectsChain().lowshelf(gain=-12.0, frequency=threshold_l, slope=0.5).highshelf(gain=-12.0, frequency=threshold_h, slope=0.5).limiter(gain=6.0)
y_cleaned = less_noise(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_cleaned,sr=sr, axis='time');
IPython.display.Audio(data=y_cleaned, rate=22055)

In [ ]:
calculate_snr(y, y_cleaned)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=y_cleaned, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

In [ ]:
cent = librosa.feature.spectral_centroid(y=y, sr=sr)
threshold_h = np.max(cent)
threshold_l = np.min(cent)
less_noise = AudioEffectsChain().lowshelf(gain=-30.0, frequency=threshold_l, slope=0.5).highshelf(gain=-30.0, frequency=threshold_h, slope=0.5).limiter(gain=10.0)
# less_noise = AudioEffectsChain().lowpass(frequency=threshold_h).highpass(frequency=threshold_l)
y_cleaned = less_noise(y)
cent_cleaned = librosa.feature.spectral_centroid(y=y_cleaned, sr=sr)
columns, rows = cent_cleaned.shape
boost_h = math.floor(rows/3*2)
boost_l = math.floor(rows/6)
boost = math.floor(rows/3)
# boost_bass = AudioEffectsChain().lowshelf(gain=20.0, frequency=boost, slope=0.8)
boost_bass = AudioEffectsChain().lowshelf(gain=16.0, frequency=boost_h, slope=0.5)#.lowshelf(gain=-20.0, frequency=boost_l, slope=0.8)
y_clean_boosted = boost_bass(y_cleaned)


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_clean_boosted,sr=sr, axis='time');
IPython.display.Audio(data=y_clean_boosted, rate=22055)

In [ ]:
calculate_snr(y, y_clean_boosted)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=y_clean_boosted, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **MFCC Denoise**
receives an audio matrix,
    returns the matrix after gain reduction on noise

In [ ]:
#mfcc_down
hop_length = 512
mfcc = python_speech_features.base.mfcc(y)
mfcc = python_speech_features.base.logfbank(y)
mfcc = python_speech_features.base.lifter(mfcc)
sum_of_squares = []
index = -1
for r in mfcc:
    sum_of_squares.append(0)
    index = index + 1
    for n in r:
        sum_of_squares[index] = sum_of_squares[index] + n**2

strongest_frame = sum_of_squares.index(max(sum_of_squares))
hz = python_speech_features.base.mel2hz(mfcc[strongest_frame])
max_hz = max(hz)
min_hz = min(hz)
speech_booster = AudioEffectsChain().highshelf(frequency=min_hz*(-1)*1.2, gain=-12.0, slope=0.6).limiter(gain=8.0)
y_speach_boosted = speech_booster(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y_speach_boosted,sr=sr, axis='time');
librosa.display.waveshow(y,sr=sr, axis='time');
IPython.display.Audio(data=y_speach_boosted, rate=22055)

In [ ]:
calculate_snr(y, y_speach_boosted)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=y_speach_boosted, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

In [ ]:
#mfcc_up
hop_length = 512
mfcc = python_speech_features.base.mfcc(y)
mfcc = python_speech_features.base.logfbank(y)
mfcc = python_speech_features.base.lifter(mfcc)
sum_of_squares = []
index = -1
for r in mfcc:
    sum_of_squares.append(0)
    index = index + 1
    for n in r:
        sum_of_squares[index] = sum_of_squares[index] + n**2
strongest_frame = sum_of_squares.index(max(sum_of_squares))
hz = python_speech_features.base.mel2hz(mfcc[strongest_frame])
max_hz = max(hz)
min_hz = min(hz)
speech_booster = AudioEffectsChain().lowshelf(frequency=min_hz*(-1), gain=12.0, slope=0.5)#.highshelf(frequency=min_hz*(-1)*1.2, gain=-12.0, slope=0.5)#.limiter(gain=8.0)
y_speach_boosted = speech_booster(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y_speach_boosted,sr=sr, axis='time');
librosa.display.waveshow(y,sr=sr, axis='time');
IPython.display.Audio(data=y_speach_boosted, rate=22055)

In [ ]:
calculate_snr(y, y_speach_boosted)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=y_speach_boosted, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **Median Denoise**
receives an audio matrix,
    returns the matrix after gain reduction on noise

In [ ]:
mod_y = sp.signal.medfilt(y,3)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(mod_y,sr=sr, axis='time');
IPython.display.Audio(data=mod_y, rate=22055)

In [ ]:
calculate_snr(y, mod_y)

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=mod_y, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel')

# **SILENCE TRIMMER**
receives an audio matrix,
    returns an audio matrix with less silence and the amout of time that was trimmed

In [ ]:
y_trimmed, index = librosa.effects.trim(y, top_db=20, frame_length=2, hop_length=500)
#trimmed_length = librosa.get_duration(y) - librosa.get_duration(y_trimmed)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_trimmed,sr=sr, axis='time');
IPython.display.Audio(data=y_trimmed, rate=22055)

In [ ]:
calculate_snr(y, y_trimmed)

# **AUDIO ENHANCER**
receives an audio matrix,
    returns the same matrix after audio manipulation

In [ ]:
apply_audio_effects = AudioEffectsChain().lowshelf(gain=10.0, frequency=260, slope=0.1).reverb(reverberance=25, hf_damping=5, room_scale=5, stereo_depth=50, pre_delay=20, wet_gain=0, wet_only=False)#.normalize()
y_enhanced = apply_audio_effects(y)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y_enhanced,sr=sr, axis='time');
librosa.display.waveshow(y,sr=sr, axis='time');
IPython.display.Audio(data=y_enhanced, rate=22055)

In [ ]:
calculate_snr(y, y_enhanced)

# **Wavelet Denoise**

In [ ]:
wavelet = 'db1'  # Daubechies wavelet with 1 vanishing moment (change as needed)
level = 4  # Adjust the level of decomposition (change as needed)
coeffs = pywt.wavedec(y, wavelet, level=level)
threshold = 0.3  # Adjust the threshold value (change as needed)
coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
y_wavelets = pywt.waverec(coeffs_thresholded, wavelet)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_wavelets,sr=sr, axis='time');
IPython.display.Audio(data=y_wavelets, rate=22055)

In [ ]:
calculate_snr(y, y_wavelets)

In [ ]:
wavelet = 'sym8'  # Daubechies wavelet with 1 vanishing moment (change as needed)
level = 5  # Adjust the level of decomposition (change as needed)
coeffs = pywt.wavedec(y, wavelet, level=level)
threshold = 0.3  # Adjust the threshold value (change as needed)
coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
y_wavelets = pywt.waverec(coeffs_thresholded, wavelet)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
librosa.display.waveshow(y,sr=sr, axis='time');
librosa.display.waveshow(y_wavelets,sr=sr, axis='time');
IPython.display.Audio(data=y_wavelets, rate=22055)

In [ ]:
calculate_snr(y, y_wavelets)

# **webrtcvad**
VAD denoise

In [ ]:
def plot_waveforms(original_audio, denoised_audio, sample_rate, title="Waveform Comparison"):
    # Convert to raw numpy arrays
    original_samples = np.array(original_audio.get_array_of_samples())
    denoised_samples = np.array(denoised_audio.get_array_of_samples())

    # Make sure both arrays have the same length
    min_length = min(len(original_samples), len(denoised_samples))
    original_samples = original_samples[:min_length]
    denoised_samples = denoised_samples[:min_length]

    # Time axis
    time = np.arange(0, min_length) / sample_rate

    # Plot original waveform
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.plot(time, original_samples, label='Original')
    #plt.title('Original Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()

    # Plot denoised waveform
    plt.subplot(2, 1, 2)
    plt.plot(time, denoised_samples, label='Denoised', color='orange')
    plt.title('Denoised Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()

    # Show the plot
    plt.tight_layout()
    plt.suptitle(title, fontsize=16)
    plt.show()
    display(Audio(denoised_samples, rate=sample_rate))

In [ ]:
import collections
import contextlib
import sys
import wave
import webrtcvad


def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])


def main(args):
    if len(args) != 2:
        sys.stderr.write(
            'Usage: example.py 3 /content/drive/MyDrive/F03.wav \n')
        sys.exit(1)
    audio, sample_rate = read_wave(args[1])
    vad = webrtcvad.Vad(int(args[0]))
    frames = frame_generator(30, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, 30, 300, vad, frames)
    for i, segment in enumerate(segments):
        path = 'chunk-%002d.wav' % (i,)
        print(' Writing %s' % (path,))
        write_wave(path, segment, sample_rate)


if __name__ == '__main__':
    main(sys.argv[1:])